# Assignment 1: Decision Trees

## 1 Packages

In [1]:
import numpy as np
import pandas as pd
from public_tests import *
from utils import *
%matplotlib inline

## 2 Problem Statement

## 3 Dataset

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
print("First few elements of X_train:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Type of X_train: <class 'numpy.ndarray'>


In [4]:
print("First few elements of y_train:", y_train[:5])
print("Type of y_train:",type(y_train))

First few elements of y_train: [1 1 0 0 1]
Type of y_train: <class 'numpy.ndarray'>


In [5]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


## 4 Decision Tree Refresher

### 4.1 Calculate entropy

### <span style="color:red">Exercise 1</span>

In [9]:
def compute_entropy(y):
    entropy=0
    if len(y)!=0:
        p1=sum(y)/len(y)
        if (p1==1 or p1==0 ):
            return 0
        else:
            entropy=-p1*np.log2(p1)-(1-p1)*np.log2(1-p1)
    return entropy

In [10]:
print("Entropy at root node: ", compute_entropy(y_train)) 

Entropy at root node:  1.0


In [11]:
compute_entropy_test(compute_entropy)

 All tests passed. 


### 4.2 Split dataset

### <span style="color:red">Exercise 2</span>

In [12]:
def split_dataset(X,node_indices,feature):
    left_indices=[]
    right_indices=[]

    for index in node_indices:
        if X[index][feature]==1:
            left_indices.append(index)
        elif X[index][feature]==0:
            right_indices.append(index)
    
    return left_indices,right_indices

In [14]:
# Case 1

root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("CASE 1:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# Visualize the split 
#generate_split_viz(root_indices, left_indices, right_indices, feature)

CASE 1:
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


In [15]:
# Case 2

root_indices_subset = [0, 2, 4, 6, 8]
left_indices, right_indices = split_dataset(X_train, root_indices_subset, feature)

print("CASE 2:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

CASE 2:
Left indices:  [0, 2, 4]
Right indices:  [6, 8]


In [16]:
# UNIT TESTS    
split_dataset_test(split_dataset)

 All tests passed.


### <span style="color:red">Exercise 3</span>

In [22]:
def compute_information_gain(X,y,node_indices,feature):
    left_indices,right_indices=split_dataset(X,node_indices,feature)
    
    X_node,y_node=X[node_indices],y[node_indices]
    X_left,y_left=X[left_indices],y[left_indices]
    X_right,y_right=X[right_indices],y[right_indices]
    
    information_gain=0
    
    node_entropy=compute_entropy(y_node)
    left_entropy=compute_entropy(y_left)
    w_left=len(X_left)/len(X_node)
    right_entropy=compute_entropy(y_right)
    w_right=len(X_right)/len(X_node)

    information_gain=node_entropy-(w_left*left_entropy+w_right*right_entropy)
    
    return information_gain

In [23]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


In [24]:
compute_information_gain_test(compute_information_gain)

 All tests passed.


### 4.4 Get best split

### <span style="color:red">Exercise 4</span>

In [25]:
def get_best_split(X,y,node_indices):
    num_features=X.shape[1]
    best_feature=-1
    max_information_gain=0

    for feature in range(num_features):
        information_gain=compute_information_gain(X,y,node_indices,feature)
        if max_information_gain<information_gain:
            max_information_gain=information_gain
            best_feature=feature
    return best_feature

In [26]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In [27]:
root_indices

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [28]:
get_best_split_test(get_best_split)

 All tests passed.


## 5 Building the tree

In [29]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [30]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]


In [31]:
#generate_tree_viz(root_indices, y_train, tree)

FileNotFoundError: [WinError 2] "dot" not found in path.